# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
normed_mpia_authors = [k[1] for k in mpia_authors]   # initials + fullname
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [mpia.get_initials(k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, normed_mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

Arxiv has 56 new papers today
          3 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [4]:
documents = []
failed = []
for paper in tqdm(candidates[:-1]):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [mpia.get_initials(k) for k in doc.authors], 
                normed_mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/2 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2304.01234


extracting tarball to tmp_2304.01234...

 done.
Retrieving document from  https://arxiv.org/e-print/2304.01718


/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3505: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3505: LatexWarning: Found documentclass in tmp_2304.01234/_manuscript__solar_wind_speed_prediction.tex

  exec(code_obj, self.user_global_ns, self.user_ns)


extracting tarball to tmp_2304.01718...

 done.


/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3505: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3505: LatexWarning: Found documentclass in tmp_2304.01718/dynamic.tex

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:399: LatexWarning: Latex injecting: 'author' from 'tmp_2304.01718/author.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))


Found 129 bibliographic references in tmp_2304.01718/dynamic.bbl.
syntax error in line 606: '}' expected


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [5]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2304.01718-b31b1b.svg)](https://arxiv.org/abs/arXiv:2304.01718) | **The ALMA Survey of 70 μm Dark High-mass Clumps in Early Stages  (ASHES). VIII. Dynamics of Embedded Dense Cores**  |
|| <mark>S. Li</mark>, et al. |
|*Appeared on*| *2023-04-05*|
|*Comments*| *29 pages, 14 figures, 5 tables. Accepted for publication by ApJ. Tables 2 and 3 are available here: this https URL*|
|**Abstract**| We present the dynamical properties of 294 cores embedded in twelve IRDCs observed as part of the ASHES Survey. Protostellar cores have higher gas masses, surface densities, column densities, and volume densities than prestellar cores, indicating core mass growth from the prestellar to the protostellar phase. We find that ~80% of cores with virial parameter ($\alpha$) measurements are gravitationally bound ($\alpha$< 2). We also find an anti-correlation between the mass and the virial parameter of cores, with massive cores having on average lower virial parameters. Protostellar cores are more gravitationally bound than prestellar cores, with an average virial parameter of 1.2 and 1.5, respectively. The observed non-thermal velocity dispersion (from N$_{2}$D$^{+}$ or DCO$^{+}$) is consistent with simulations in which turbulence is continuously injected, whereas the core-to-core velocity dispersion is neither in agreement with driven nor decaying turbulence simulations. We find no significant increment in the line velocity dispersion from prestellar to protostellar cores, suggesting that dense gas within the core traced by these deuterated molecules is not yet severely affected by turbulence injected from outflow activity at the early evolutionary stages traced in ASHES. The most massive cores are strongly self-gravitating and have greater surface density, Mach number, and velocity dispersion than cores with lower masses. Dense cores have not significant velocity shifts relative to their low-density envelopes, suggesting that dense cores are co-moving with their envelopes. We conclude that the observed core properties are more in line with predictions of ``clump-fed" scenarios rather than with ``core-fed" scenarios. |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2304.01234-b31b1b.svg)](https://arxiv.org/abs/arXiv:2304.01234) | **Prediction of solar wind speed by applying convolutional neural network  to potential field source surface (PFSS) magnetograms**  |
|| R. Lin, et al. -- incl., <mark>J. He</mark> |
|*Appeared on*| *2023-04-05*|
|*Comments*| **|
|**Abstract**| An accurate solar wind speed model is important for space weather predictions, catastrophic event warnings, and other issues concerning solar wind - magnetosphere interaction. In this work, we construct a model based on convolutional neural network (CNN) and Potential Field Source Surface (PFSS) magnetograms, considering a solar wind source surface of $R_{\rm SS}=2.5R_\odot$, aiming to predict the solar wind speed at the Lagrange 1 (L1) point of the Sun-Earth system. The input of our model consists of four Potential Field Source Surface (PFSS) magnetograms at $R_{\rm SS}$, which are 7, 6, 5, and 4 days before the target epoch. Reduced magnetograms are used to promote the model's efficiency. We use the Global Oscillation Network Group (GONG) photospheric magnetograms and the potential field extrapolation model to generate PFSS magnetograms at the source surface. The model provides predictions of the continuous test dataset with an averaged correlation coefficient (CC) of 0.52 and a root mean square error (RMSE) of 80.8 km/s in an eight-fold validation training scheme with the time resolution of the data as small as one hour. The model also has the potential to forecast high speed streams of the solar wind, which can be quantified with a general threat score of 0.39. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [6]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [7]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2304.01718.md
    + _build/html/tmp_2304.01718/figures/vir_hist.png
    + _build/html/tmp_2304.01718/figures/virial.png
    + _build/html/tmp_2304.01718/figures/sig_R_core_clump3.png
    + _build/html/tmp_2304.01718/figures/sig_R_core_clump2.png
    + _build/html/tmp_2304.01718/figures/sig_R_core_clump1.png
    + _build/html/tmp_2304.01718/figures/vir_R.png


## Display the papers

Not necessary but allows for a quick check.

In [8]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand{\HII}{H{\scriptsize II}\xspace}$
$\newcommand{\um}{{\mum}\xspace}$
$\newcommand$
$\newcommand{\SH}[1]{{\color{red}  (SH: #1)}}$
$\newcommand{\PS}[1]{{\color{green}  PS: #1}}$
$\newcommand{\QZcomment}[1]{{\color{red}  (QZ; #1)}}$
$\newcommand{\arcsec}{^{\prime\prime}\xspace}$
$\newcommand{\Mo}{M_{\odot}\xspace}$
$\newcommand{\kms}{km~s^{-1}\xspace}$
$\newcommand{\co18}{C^{18}O}$
$\newcommand{\dcop}{DCO^{+}\xspace}$
$\newcommand{\n2dp}{N_{2}D^{+}}$
$\newcommand{çd}{C_{2}D\xspace}$
$\newcommand{\ch3oh}{CH_{3}OH}$
$\newcommand{\h2co}{H_{2}CO}$
$\newcommand{\1}{\uppercase\expandafter{\romannumeral1}}$
$\newcommand{\2}{\uppercase\expandafter{\romannumeral2}}$
$\newcommand{\3}{\uppercase\expandafter{\romannumeral3}}$
$\newcommand{\6}{\uppercase\expandafter{\romannumeral6}}$
$\newcommand{\7}{\uppercase\expandafter{\romannumeral7}}$
$\newcommand{\8}{\uppercase\expandafter{\romannumeral8}}$
$\newcommand{\9}{\uppercase\expandafter{\romannumeral9}}$</div>



<div id="title">

# The ALMA Survey of 70 $\mu$m Dark High-mass Clumps in Early Stages (ASHES). \8. Dynamics of Embedded Dense Cores

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2304.01718-b31b1b.svg)](https://arxiv.org/abs/2304.01718)<mark>Appeared on: 2023-04-05</mark> -  _29 pages, 14 figures, 5 tables. Accepted for publication by ApJ. Tables 2 and 3 are available here: this https URL_

</div>
<div id="authors">

<mark>S. Li</mark>, et al.

</div>
<div id="abstract">

**Abstract:** We present the dynamical properties of 294 coresembedded in twelve IRDCsobserved as part of the ASHES Survey.Protostellar cores have higher gas masses,surface densities, column densities, and volumedensities than prestellar cores, indicatingcore mass growth from the prestellar to the protostellarphase. We find that $\sim$ 80 \% of cores with virialparameter ( $\alpha$ ) measurements are gravitationallybound ( $\alpha<$ 2).  We also find an anticorrelationbetween the mass and the virial parameter of cores,with massive cores having on average lower virialparameters. Protostellar cores are more gravitationallybound than prestellar cores, with an average virialparameter of 1.2 and 1.5, respectively.The observed nonthermal velocity dispersion(from  N $_{2}$ D $^{+}$ or DCO $^{+}$ )is consistent with simulations in whichturbulence is continuously injected, whereas thecore-to-core velocity dispersion isneither in agreement with driven nor decaying turbulencesimulations. We find a no significant increment in the linevelocity dispersion from prestellar to protostellar cores,suggesting that the dense gas within the core tracedby these deuterated molecules is not yet severely affected byturbulence injected from outflow activity at the earlyevolutionary stages traced in ASHES. The most massivecores are strongly self-gravitating and havegreater surface density, Mach number, and velocitydispersion than cores with lower masses.Dense cores do not have significant velocity shiftsrelative to their low-density envelopes, suggesting that dense cores are comoving with their envelopes. We conclude that the observed core properties are more in line with the predictions of "clump-fed" scenarios rather than those of "core-fed" scenarios.

</div>

<div id="div_fig1">

<img src="tmp_2304.01718/figures/vir_hist.png" alt="Fig6.1" width="50%"/><img src="tmp_2304.01718/figures/virial.png" alt="Fig6.2" width="50%"/>

**Figure 6. -** 
Panel (a): upper panel shows the number of
sources per mass bin, and the bottom panel shows
a box plot of the values of the virial parameter per bin.
In the box plot, the mean value of the virial parameter
is shown with a horizontal line.
Panels (b) and (c): $\alpha$ versus mass for all the
cores where molecular emission from either $\n$2dp
or $\dcop$ was detected. Panel (b) shows the
cores color coded by their evolutionary stage
classification \citep{Li-2022b}, and panel (c) shows the
cores color coded by their parental molecular cloud.
The solid line in both figures shows the linear
regression to all the point in the plot, gives a slope
of -0.61 $\pm$ 0.06, and the gray shadowed area
shows the 1$\sigma$ confidence interval
for the fit. The Spearman's rank test returns a
coefficient of $r_{s}$ = -0.68.
 (*fig:alpha*)

</div>
<div id="div_fig2">

<img src="tmp_2304.01718/figures/sig_R_core_clump3.png" alt="Fig1.1" width="33%"/><img src="tmp_2304.01718/figures/sig_R_core_clump2.png" alt="Fig1.2" width="33%"/><img src="tmp_2304.01718/figures/sig_R_core_clump1.png" alt="Fig1.3" width="33%"/>

**Figure 1. -** Observed velocity dispersion $\sigma_{\rm obs}$
as a function of the radius.
From upper to bottom, panels show the
$\sigma_{\rm obs}$ versus radius for dense cores,
clumps and dense core, and GMCs down to core,
respectively.
The blue solid line shows the linear regression to all
the points in the plot,  and the gray shadowed
area shows the 1$\sigma$ confidence interval for the fit.
The best fit returns slopes of 0.24 $\pm$ 0.08,
0.46 $\pm$ 0.01, and 0.35 $\pm$ 0.01 for upper,
middle, and bottom panels, respectively.
The Spearman's rank coefficients between observed
velocity and radius are presented in the lower right
corner in each panel.
The black dashed line shows the the original Larson
relation with  $\sigma_{\rm obs} \propto R^{0.38}$\citep{Larson-1981}, and the red
dashed line is the revised \cite{Heyer-2004} relation
with $\sigma_{\rm obs} \propto R^{0.56}$.
 (*fig:sigrad*)

</div>
<div id="div_fig3">

<img src="tmp_2304.01718/figures/vir_R.png" alt="Fig7" width="100%"/>

**Figure 7. -** Left panel: virial parameter versus surface
density.
Right panel: virial parameter versus peak column
density.
The blue solid line in both figures shows the linear
regression to all the point in the plot, and the gray
shadowed area shows the 1$\sigma$ confidence interval
for the fit. The best fit gives a slope
-1.22 $\pm$ 0.15 and -1.17 $\pm$ 0.13 for
$\alpha$--$\Sigma$ and $\alpha$--$N_{\rm H_2}$,
respectively.
The Spearman's rank coefficients are shown in the
lower left of each panel.
 (*fig:alpha1*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2304.01718"></div>

# Create HTML index

In [9]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

183  publications files modified in the last 7 days.


In [10]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

7  publications in the last 7 days.


In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [13]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

2  publications in the last day.


In [14]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
